In [1]:
import pandas as pd
import math
import ast
from datetime import datetime
import copy

In [2]:
df = pd.read_csv(r'/Users/sara/Downloads/funds data cleaned.csv')


In [3]:
df.head()

,id,asset_class,aum,geographic_focus,geography,impact,inception_date,name,status,strategy,summary,ticker,type,impact_category_ids
0,d8df96e6-8450-4bc4-b251-60c0cf8b407d,"[""Fixed Income""]","{""aums"": [{""date"": ""2016-06-30"", ""value"": 3325...","[""US""]","{""primary"": {""scope"": ""Country-Focus"", ""market...","{""focus"": {""food"": false, ""audit"": false, ""lab...",1/1/1970,Horizon Enhanced SRI Fossil Fuel Free,"{""accepting_erisa_funds"":false,""accepting_fund...","{""value"": [""Core""]}",The Enhanced SRI Fossil Fuel Free strategy is ...,HEFFLOD,Open-End Fund,"[""fc978270-74c4-40d4-a3ca-42c537072fa2""]"
1,bdd1f532-6c05-40b4-a33e-fd9828eb7c4c,"[""Equity""]","{""aums"": [{""date"": ""2016-06-30"", ""value"": 2557...","[""Global""]","{""primary"": {""scope"": ""Global"", ""market"": [""De...","{""focus"": {""food"": false, ""audit"": false, ""lab...",6/4/2013,HSBC Amanah Global Equity Index Fund,"{""accepting_erisa_funds"":false,""accepting_fund...",NaN,NaN,AGEILOF,Index,"[""fc978270-74c4-40d4-a3ca-42c537072fa2""]"
2,00336ba9-9235-4bea-8677-2a6875e4bd46,"[""Equity""]","{""aums"": [{""date"": ""2016-06-30"", ""value"": 6363...","[""US""]","{""primary"": {""scope"": ""Country-Focus"", ""market...","{""focus"": {""food"": false, ""audit"": false, ""lab...",4/1/2008,Pax Small Cap Fund,"{""accepting_erisa_funds"":false,""accepting_fund...","{""aum"": {""value"": {""value"": 703830020, ""curren...",NaN,PWSCLOD,Open-End Fund,"[""fc978270-74c4-40d4-a3ca-42c537072fa2""]"
3,00c8c91c-7b4e-4e7f-a227-47b1e3b4d228,"[""Equity""]","{""aums"": [{""date"": ""2015-03-31"", ""value"": 1855...","[""Global""]","{""primary"": {""scope"": ""Global"", ""market"": [""De...","{""focus"": {""food"": false, ""audit"": false, ""lab...",1/1/2015,KBI Emerging Markets ESG Equity Strategy,"{""accepting_erisa_funds"":false,""accepting_fund...","{""aum"": {""value"": {""value"": 26500000, ""currenc...",The KBI Emerging Markets ESG Equity Strategy i...,KEMELO,Open-End Fund,"[""fc978270-74c4-40d4-a3ca-42c537072fa2""]"
4,07887913-b05b-45ae-9753-b6aa9865c5ea,"[""Fixed Income""]","{""aums"": [{""date"": ""2016-06-30"", ""value"": 4610...","[""US""]","{""primary"": {""scope"": ""Country-Focus"", ""market...","{""focus"": {""food"": false, ""audit"": false, ""lab...",1/1/1970,Knights of Columbus Core Bond Fund,"{""accepting_erisa_funds"":false,""accepting_fund...","{""value"": [""Core""]}","Under normal circumstances, the Fund invests a...",KCCBLOD,Open-End Fund,"[""fc978270-74c4-40d4-a3ca-42c537072fa2""]"


In [4]:
## Adding AUM values

dfcleaned = df


aum_values = []

for i in range(len(df)):
    x1 = df.iloc[i]['aum'].find("value") + 8
    x2 = df.iloc[i]['aum'].find("}")
    aum_values.append(int(float(df.iloc[i]['aum'][x1:x2])))
    
dfcleaned['AUM'] = aum_values
dfcleaned.drop(columns = ['aum'])

assetclasses = []
impact_categories = []
for i in range(len(dfcleaned)):
    if not(isinstance(dfcleaned.iloc[i]['impact_category_ids'],str)):
        categories = []
    else:
        categories = ast.literal_eval(dfcleaned.iloc[i]['impact_category_ids'])

        
    assetclasses.append(ast.literal_eval(dfcleaned.iloc[i]['asset_class']))
    impact_categories.append(categories)
    



    
    
dfcleaned['impact_tags'] = impact_categories
dfcleaned['asset_class'] = assetclasses
dfcleaned.drop(columns = ['impact_category_ids'],inplace = True)

tobedeleted = []
for i in range(len(dfcleaned)):
    if (not dfcleaned.iloc[i]['impact_tags']) or (dfcleaned.iloc[i]['AUM'] == 0):
        tobedeleted.append(i)     

dfcleaned.drop(tobedeleted,inplace=True)

In [5]:
##Cleaning Strategies
strategies = []
for i in range(len(dfcleaned)):
    if not(isinstance(dfcleaned.iloc[i]['strategy'],str)):
        currstrat = []
    else:
        if 'value' not in ast.literal_eval(dfcleaned.iloc[i]['strategy']):
            currstrat = []
        else:currstrat = ast.literal_eval(dfcleaned.iloc[i]['strategy'])['value']
    strategies.append(currstrat)
    
dfcleaned['strat'] = strategies    

In [6]:
##Adding Strategies
fixedIncomeFeature = []
for i in range(len(dfcleaned)):
    if 'Fixed Income' in dfcleaned.iloc[i]['asset_class']:
        fixedIncomeFeature.append(1)
    else:
        fixedIncomeFeature.append(0)
dfcleaned['isFixedIncome'] =  fixedIncomeFeature  

EquityFeature = []
for i in range(len(dfcleaned)):
    if 'Equity' in dfcleaned.iloc[i]['asset_class']:
        EquityFeature.append(1)
    else:
        EquityFeature.append(0)
dfcleaned['isEquity'] =  EquityFeature  

AlternativeFeature = []
for i in range(len(dfcleaned)):
    if 'Alternative' in dfcleaned.iloc[i]['asset_class']:
        AlternativeFeature.append(1)
    else:
        AlternativeFeature.append(0)
dfcleaned['isAlternative'] =  AlternativeFeature  

HedgefundFeature = []
for i in range(len(dfcleaned)):
    if 'Hedge Fund' in dfcleaned.iloc[i]['asset_class']:
        HedgefundFeature.append(1)
    else:
        HedgefundFeature.append(0)
dfcleaned['isHedgefund'] =  HedgefundFeature  

realEstateFeature = []
for i in range(len(dfcleaned)):
    if 'Real Estate' in dfcleaned.iloc[i]['asset_class']:
        realEstateFeature.append(1)
    else:
        realEstateFeature.append(0)
dfcleaned['isRealEstate'] =  realEstateFeature  

dfcleaned.drop(columns = ['asset_class', 'aum'],inplace =True)

In [7]:
#cleaning impact, and adding impact
impact_to_id = {}
geographic_cat = ['US', 'Global', 'Emerging Markets', 'Developed Markets', 'Euro-Zone', 'Canada', 'Asia', 'Japan']

for cat in geographic_cat:
    
    newcol = []

    for i in range(len(dfcleaned)):
        if cat in dfcleaned.iloc[i]['geographic_focus']:
            newcol.append(1)
        else:
            newcol.append(0)
    dfcleaned['GF_' + cat] =  newcol 
    


In [8]:
#adding inception
inceptionyear = []
for i in range(len(dfcleaned)):
    date_object = datetime.strptime(dfcleaned.iloc[i]['inception_date'], '%m/%d/%Y') 
    inceptionyear.append(date_object.year)
    
dfcleaned['inception_date'] = inceptionyear

In [9]:
#adding geography
geographic_cat = ['US', 'Global', 'Emerging Markets', 'Developed Markets', 'Euro-Zone', 'Canada', 'Asia', 'Japan']

for cat in geographic_cat:
    
    newcol = []

    for i in range(len(dfcleaned)):
        if cat in dfcleaned.iloc[i]['geographic_focus']:
            newcol.append(1)
        else:
            newcol.append(0)
    dfcleaned['GF_' + cat] =  newcol 




In [10]:
#impact tags
impacttags = []
for i in range(len(dfcleaned)):
    for tag in dfcleaned.iloc[i]['impact_tags']:
        if tag not in impacttags:
            impacttags.append(tag)


In [11]:
#adding and cleaning tags
j = 1
for tag in impacttags:
    
    newcol = []
    for i in range(len(dfcleaned)):
        if cat in dfcleaned.iloc[i]['impact_tags']:
            newcol.append(1)
        else:
            newcol.append(0)
    dfcleaned['impact tag' + str(j)] =  newcol 
    j+=1


In [12]:
#strategies
strategies = []
for i in range(len(dfcleaned)):
    for st in dfcleaned.iloc[i]['strat']:
        if st not in strategies:
            strategies.append(st)
            
print(strategies)

['Core', 'Diversified', 'Long Only', 'Equity Hedge', 'Event Driven', 'Equity Market Neutral', 'Sector Focus', 'Balanced', 'Relative Value']


In [13]:
j = 1
for st in strategies:
    
    newcol = []
    for i in range(len(dfcleaned)):
        if st in dfcleaned.iloc[i]['strat']:
            newcol.append(1)
        else:
            newcol.append(0)
    dfcleaned['strat_' + st] =  newcol 
    j+=1


In [14]:
types = []
for i in range(len(dfcleaned)):
    if dfcleaned.iloc[i]['type'] not in types:
        types.append(dfcleaned.iloc[i]['type'])
            
print(types)
types.pop(2)

j = 1
for ft in types:
    newcol = []
    for i in range(len(dfcleaned)):
        if dfcleaned.iloc[i]['type'] == ft:
            newcol.append(1)
        else:
            newcol.append(0)
    dfcleaned['type_' + ft] =  newcol 
    j+=1

['Open-End Fund', 'Index', nan, 'Security', 'Hedge Fund', 'ETF']


In [15]:
dfbinary = copy.deepcopy(dfcleaned)
dfqual = copy.deepcopy(dfcleaned)
dfbinary.drop(columns = ['name', 'id','geographic_focus','geography','impact', 'status', 'strategy', 'summary', 'ticker', 'type', 'impact_tags', 'strat' ], inplace = True)

In [16]:
dfqual.drop(columns = [ 'geography','strategy','summary','status', 'ticker', 'impact', 'isFixedIncome', 'isEquity', 'isAlternative',
       'isHedgefund', 'isRealEstate', 'GF_US', 'GF_Global',
       'GF_Emerging Markets', 'GF_Developed Markets', 'GF_Euro-Zone',
       'GF_Canada', 'GF_Asia', 'GF_Japan', 'impact tag1', 'impact tag2',
       'impact tag3', 'impact tag4', 'impact tag5', 'impact tag6',
       'impact tag7', 'impact tag8', 'impact tag9', 'impact tag10',
       'impact tag11', 'impact tag12', 'impact tag13', 'impact tag14',
       'impact tag15', 'strat_Core', 'strat_Diversified', 'strat_Long Only',
       'strat_Equity Hedge', 'strat_Event Driven',
       'strat_Equity Market Neutral', 'strat_Sector Focus', 'strat_Balanced',
       'strat_Relative Value', 'type_Open-End Fund', 'type_Index',
       'type_Security', 'type_Hedge Fund', 'type_ETF'],inplace = True)

In [17]:
from sklearn.cluster import KMeans

In [18]:
import numpy as np
import math
from collections import Counter

In [19]:
matrix = copy.deepcopy(dfbinary.values)
matrix_scaled=np.zeros(matrix.shape)


matrix_scaled[:,0] = (matrix[:,0] - matrix[:,0].min(0)) / matrix[:,0].ptp(0)
print(matrix[:,1])
matrix[:,1] = np.log10(matrix[:,1]) 
print(matrix[:,1])
matrix_scaled[:,1] = (matrix[:,1] - matrix[:,1].min(0)) / matrix[:,1].ptp(0)


for i in range(2,len(matrix[0])):
    matrix_scaled[:,i] = matrix[:,i]
    
print(matrix_scaled)

[    3325000   255700000   636360000    18559000    46100000     3670000
    21100000    21500000   220000000  2696000000     4770000    64070000
   502834624   137188096     2940000       50000   189700000   321500000
   423929984    10560000    25507000   288000000    25757452     5620000
   364000000     1070000   192600000  1347000060   101160000     2600000
   117460000     3480000     9000000   199500000   532000000   168910000
   270000000   740200000    10466000    13546000   195180000   203800000
   236120000   267600000     8000000  2268940030   457600000   320148320
    51000000     7000000    39200000   113000000     1010000   743960640
    24746124    92080000    14337256   393449984     1530000     2720000
     5940000    75000000    46790000     8680000    24976320     1394000
    24500000     8510000   157237248    19700000   521895136     6000000
    24190000      250000     7790000     4380000     5100907    59000000
       11000      252779     2460000    36482000   

In [20]:
kmeans = KMeans(n_clusters=10, init='k-means++').fit(matrix_scaled)

In [21]:
kmeans.cluster_centers_


array([[ 7.67784553e-01,  5.04065041e-01,  0.00000000e+00,
         1.00000000e+00,  5.55111512e-17, -1.04083409e-17,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         4.16333634e-17, -3.46944695e-17, -1.04083409e-17,
         8.67361738e-19,  1.73472348e-18,  8.67361738e-19,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.38777878e-16,  9.75609756e-01,  3.46944695e-18,
         6.93889390e-18,  1.73472348e-18,  1.73472348e-18,
        -1.04083409e-17,  8.67361738e-19,  8.67361738e-19,
         9.75609756e-01,  2.43902439e-02,  8.67361738e-19,
        -4.16333634e-17,  1.73472348e-18],
       [ 7.76041667e-01,  5.41666667e-01,  8.33333333e-02,
         9.16666667e-01,  1.00000000e+00,  0.00000000e+00,
         0.00

In [22]:
labels = kmeans.predict(matrix_scaled)

In [23]:
labelscopy = np.asarray(labels)
populations = Counter()
for i in range(len(labelscopy)):
    populations[labelscopy[i]] += 1
    
print(labels)
print(populations)

[2 0 3 0 2 2 0 2 2 3 4 2 0 0 0 2 3 3 0 8 2 0 5 0 2 8 2 7 4 3 3 0 2 0 6 3 5
 3 0 2 3 3 8 3 7 8 0 4 2 0 2 2 2 2 0 0 0 8 2 8 3 4 4 0 3 7 3 4 1 1 3 3 1 3
 9 9 0 3 3 0 0 3 9 1 0 0 0 8 1 3 3 3 9 1 4 0 3 0 1 3 2 0 0 9 0 3 0 3 3 1 4
 1 0 0 0 3 0 4 0 6 1 0 6 3 4 9 8 2 3 9 0 3 1 0 3 9 2 3 2 0 4 0 0 1]
Counter({0: 41, 3: 34, 2: 22, 1: 12, 4: 11, 8: 8, 9: 8, 7: 3, 6: 3, 5: 2})


In [24]:
for i in range(len(labelscopy)):
    if labels[i] == 0: 
        print(dfqual.iloc[i])

id                    bdd1f532-6c05-40b4-a33e-fd9828eb7c4c
geographic_focus                                ["Global"]
inception_date                                        2013
name                  HSBC Amanah Global Equity Index Fund
type                                                 Index
AUM                                              255700000
impact_tags         [fc978270-74c4-40d4-a3ca-42c537072fa2]
strat                                                   []
Name: 1, dtype: object
id                      00c8c91c-7b4e-4e7f-a227-47b1e3b4d228
geographic_focus                                  ["Global"]
inception_date                                          2015
name                KBI Emerging Markets ESG Equity Strategy
type                                           Open-End Fund
AUM                                                 18559000
impact_tags           [fc978270-74c4-40d4-a3ca-42c537072fa2]
strat                                          [Diversified]
Name: 3, dtype: o

In [25]:
dfqual.head()

,id,geographic_focus,inception_date,name,type,AUM,impact_tags,strat
0,d8df96e6-8450-4bc4-b251-60c0cf8b407d,"[""US""]",1970,Horizon Enhanced SRI Fossil Fuel Free,Open-End Fund,3325000,[fc978270-74c4-40d4-a3ca-42c537072fa2],[Core]
1,bdd1f532-6c05-40b4-a33e-fd9828eb7c4c,"[""Global""]",2013,HSBC Amanah Global Equity Index Fund,Index,255700000,[fc978270-74c4-40d4-a3ca-42c537072fa2],[]
2,00336ba9-9235-4bea-8677-2a6875e4bd46,"[""US""]",2008,Pax Small Cap Fund,Open-End Fund,636360000,[fc978270-74c4-40d4-a3ca-42c537072fa2],[Diversified]
3,00c8c91c-7b4e-4e7f-a227-47b1e3b4d228,"[""Global""]",2015,KBI Emerging Markets ESG Equity Strategy,Open-End Fund,18559000,[fc978270-74c4-40d4-a3ca-42c537072fa2],[Diversified]
4,07887913-b05b-45ae-9753-b6aa9865c5ea,"[""US""]",1970,Knights of Columbus Core Bond Fund,Open-End Fund,46100000,[fc978270-74c4-40d4-a3ca-42c537072fa2],[Core]


In [26]:
from scipy.spatial import distance

In [27]:
distance_matrix = [[distance.euclidean(matrix_scaled[i],matrix_scaled[j]) for j in range(len(matrix_scaled))] for i in range(len(matrix_scaled))]


In [28]:
distance_matrix[4]

[0.16666666666666669,
 2.7982664524467444,
 2.1574322443332696,
 2.6227669072946607,
 0.0,
 0.8907315969346646,
 2.6153924710792027,
 0.9375,
 0.5866719933470301,
 2.1279595403944023,
 2.455153310442706,
 0.8958333333333334,
 2.573317276504309,
 2.579634448694018,
 2.573317276504309,
 1.0077822185373186,
 2.006932429798716,
 2.006932429798716,
 2.579634448694018,
 1.6106470835592066,
 0.7291666666666666,
 2.6280570822905993,
 2.6153924710792027,
 2.573317276504309,
 0.3726779962499649,
 1.6713475933442983,
 0.4876958296133177,
 2.260776661041756,
 2.606415695505569,
 2.215103615585009,
 2.135289630367636,
 2.573317276504309,
 0.16666666666666669,
 2.567153633458227,
 2.7550458758027565,
 2.22400102418042,
 2.1574322443332696,
 2.206367049145621,
 2.6227669072946607,
 0.0,
 2.006932429798716,
 2.006932429798716,
 1.9293061504650375,
 2.006932429798716,
 2.4303670561826207,
 1.495508321236926,
 2.606415695505569,
 2.7727123954311916,
 0.0,
 2.606415695505569,
 0.9375,
 0.6468406123441677

In [29]:
np.argpartition(distance_matrix[4], 5)[1:6]

array([ 48,  39, 127,   0,  32])

In [30]:
print(dfqual.iloc[127])
print(dfqual.iloc[4])

id                    d1eb01ab-5220-4e28-a92b-a99cc3dc86ea
geographic_focus                                    ["US"]
inception_date                                        1970
name                    Jantz Management Fossil Free Value
type                                         Open-End Fund
AUM                                                1000000
impact_tags         [fc978270-74c4-40d4-a3ca-42c537072fa2]
strat                                               [Core]
Name: 141, dtype: object
id                    07887913-b05b-45ae-9753-b6aa9865c5ea
geographic_focus                                    ["US"]
inception_date                                        1970
name                    Knights of Columbus Core Bond Fund
type                                         Open-End Fund
AUM                                               46100000
impact_tags         [fc978270-74c4-40d4-a3ca-42c537072fa2]
strat                                               [Core]
Name: 4, dtype: object


In [31]:
def searchbyname(name):
    index = dfqual.loc[dfqual['name'] == name].index[0]
    print(index)
    relevant_indices = np.argpartition(distance_matrix[index], 5)[1:6]
    
    print(dfqual.iloc[index])
    print('Relevant Funds:')
    for x in relevant_indices:
        print(dfqual.iloc[x])

        


In [38]:
searchbyname('Parnassus Endeavor Equity')
    

29
id                               66106f47-484a-490e-a8bc-89ea11ace507
geographic_focus                                               ["US"]
inception_date                                                   2015
name                                   Etho Climate Leadership US ETF
type                                                    Open-End Fund
AUM                                                           2600000
impact_tags         [99c4500d-2540-4aed-b323-5e18114502ce, fc97827...
strat                                                   [Diversified]
Name: 31, dtype: object
Relevant Funds:
id                    6e6c0adf-6950-4c2c-bd51-7375f9598c50
geographic_focus                                    ["US"]
inception_date                                        2007
name                Martin Investment Global Eco Investing
type                                         Open-End Fund
AUM                                                1540000
impact_tags         [fc978270-74c4-40d4-a3c

In [33]:
dfbinary.iloc[0]

inception_date                    1970
AUM                            3325000
isFixedIncome                        1
isEquity                             0
isAlternative                        0
isHedgefund                          0
isRealEstate                         0
GF_US                                1
GF_Global                            0
GF_Emerging Markets                  0
GF_Developed Markets                 0
GF_Euro-Zone                         0
GF_Canada                            0
GF_Asia                              0
GF_Japan                             0
impact tag1                          0
impact tag2                          0
impact tag3                          0
impact tag4                          0
impact tag5                          0
impact tag6                          0
impact tag7                          0
impact tag8                          0
impact tag9                          0
impact tag10                         0
impact tag11             

In [34]:
def searchbycriteria(inception, AUM, otherfeatures):
    scaledinception = (inception - matrix[:,0].min(0)) / matrix[:,0].ptp(0)
    scaledAUM = (math.log10(AUM) - matrix[:,1].min(0)) / matrix[:,1].ptp(0)
    print(scaledinception)
    print(scaledAUM)
    otherfeatures.insert(0,scaledinception)
    otherfeatures.insert(1,scaledAUM)
    currlabel = kmeans.predict([otherfeatures]) 
    print(currlabel)
    counter = 0
    for i in range(len(labelscopy)):
        if counter == 5:
            break
        if labelscopy[i] == currlabel: 
            counter += 1
            print(dfqual.iloc[i])

In [35]:
searchbycriteria(1990, 12323423, [1.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0   , 0.   , 0.   , 0.   , 1   ])

0.4166666666666667
0.5151218926403951
[2]
id                    d8df96e6-8450-4bc4-b251-60c0cf8b407d
geographic_focus                                    ["US"]
inception_date                                        1970
name                 Horizon Enhanced SRI Fossil Fuel Free
type                                         Open-End Fund
AUM                                                3325000
impact_tags         [fc978270-74c4-40d4-a3ca-42c537072fa2]
strat                                               [Core]
Name: 0, dtype: object
id                    07887913-b05b-45ae-9753-b6aa9865c5ea
geographic_focus                                    ["US"]
inception_date                                        1970
name                    Knights of Columbus Core Bond Fund
type                                         Open-End Fund
AUM                                               46100000
impact_tags         [fc978270-74c4-40d4-a3ca-42c537072fa2]
strat                                             